In [5]:
import numpy as np
import pandas
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from create_train_test_val_maps import *

In [6]:
train_map_revived = open_map('/home/cs231n/data/train')
val_map_revived = open_map('/home/cs231n/data/val')
test_map_revived = open_map('/home/cs231n/data/test')
codes = [45021, 44004, 43004, 45008, 45002, 45007]

In [7]:
# Categorical Data to Numeric Format

# Train Data
category_var = ['Veh Ref ID','Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','Population','DTCID']
for ATA6code in codes:
    for i in range(len(train_map_revived[ATA6code])):
        for j in range(len(train_map_revived[ATA6code][i])):
            for k in category_var:
                train_map_revived[ATA6code][i][j][k] = train_map_revived[ATA6code][i][j][k].astype('category')
                train_map_revived[ATA6code][i][j][k] = train_map_revived[ATA6code][i][j][k].cat.codes

# Validate Data
for ATA6code in codes:
    for i in range(len(val_map_revived[ATA6code])):
        for j in range(len(val_map_revived[ATA6code][i])):
            for k in category_var:
                val_map_revived[ATA6code][i][j][k] = val_map_revived[ATA6code][i][j][k].astype('category')
                val_map_revived[ATA6code][i][j][k] = val_map_revived[ATA6code][i][j][k].cat.codes
                
                
# Test Data
for ATA6code in codes:
    for i in range(len(test_map_revived[ATA6code])):
        for j in range(len(test_map_revived[ATA6code][i])):
            for k in category_var:
                test_map_revived[ATA6code][i][j][k] = test_map_revived[ATA6code][i][j][k].astype('category')
                test_map_revived[ATA6code][i][j][k] = test_map_revived[ATA6code][i][j][k].cat.codes

In [8]:
# get global index from (row, col) index
def sub2ind(array_shape, row, col):
    ind = row*array_shape[1] + col
    if row < 0 or row >= array_shape[0]:
        ind = -1
    if col < 0 or col >= array_shape[1]:
        ind = -1
    return ind

# get (row, col) index from global index
def ind2sub(array_shape, ind):
    row = int(ind) / array_shape[1]
    col = ind % array_shape[1]
    if ind < 0:
        row = -1
        col = -1
    if ind >=  array_shape[0]*array_shape[1]:
        row = -1
        col = -1
    return (row, col)

def softmax(x):
    """
    Compute softmax function for input. 
    Use tricks from previous assignment to avoid overflow
    """
	### YOUR CODE HERE
    xshift = np.max(x, axis = 1)
    xshift = xshift.reshape((x.shape[0],1))
    x = x - xshift
    s = np.exp(x) / np.sum(np.exp(x),axis = 1).reshape((x.shape[0],1))
	### END YOUR CODE
    return s

In [ ]:
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# TRAIN
num_states = 2
num_codes = len(codes)
num_time_steps = 10
num_iter = 10
models = []
lengths = []

# For each code type
for i, ATA6code in enumerate(codes):
    # For each numer time steps  left
    for j in train_map_revived[ATA6code].keys():
        #print(j,'\n')
        X = pandas.concat(train_map_revived[ATA6code][j]).as_matrix()[:,2:] # ignore first two columns (veh id, timestamp)
        lengths = []
        # Concatenate all sequences of that code and time
        for sequence_of_snapshots in train_map_revived[ATA6code][j]:
            lengths.append(sequence_of_snapshots.shape[0])                
        # Make an HMM instance and execute fit (i.e. train)
        models.append(hmm.GaussianHMM(n_components=num_states, n_iter=num_iter).fit(X,lengths))

In [12]:
X = pandas.concat(train_map_revived[codes[0]][9]).as_matrix()[:,2:]
lengths=[]
X = []
ATA6code = 45021
j = 9
for temp in train_map_revived[ATA6code][j]:
    #temp = train_map_revived[ATA6code][j][vehicle]
    if (len(temp) > 0):
        lengths.append(len(temp))
        if (len(X) == 0):
            X = temp
        else:
            #print(temp)
            X = pandas.concat([X,temp])


temp = hmm.GaussianHMM(n_components=4, n_iter=10, covariance_type='diag').fit(X.as_matrix()[:,2:],lengths)
print(temp.startprob_)


test we're in the HMM code!!
[ nan  nan  nan  nan]
        Veh Ref ID      Event DateTime  Event Type Description  \
755268           0 2016-04-11 23:42:00                       2   
755267           0 2016-04-11 23:42:01                       0   
755270           0 2016-04-12 12:57:58                       2   
755269           0 2016-04-12 12:59:04                       0   
755248           0 2016-04-12 14:53:49                       2   
755271           0 2016-04-12 14:55:51                       0   
755265           0 2016-04-12 18:23:18                       2   
755263           0 2016-04-12 18:28:02                       0   
755257           0 2016-04-12 19:32:17                       2   
755256           0 2016-04-12 19:33:10                       0   
755261           0 2016-04-12 20:45:53                       2   
755260           0 2016-04-12 20:48:35                       0   
755266           0 2016-04-13 12:07:55                       2   
755249           0 2016-0

In [ ]:
#for i in range(len(models)):
#    #print(models[i].startprob_)
#    print(models[i].transmat_)
    
print(models[9].transmat_)
print(models[9].startprob_)

In [ ]:
# VALIDATE

# Get number of validation samples (sequences)
for i, ATA6code in enumerate(codes):
    for j in val_map_revived[ATA6code].keys():
        lengths = []
        for sequence_of_snapshots in val_map_revived[ATA6code][j]:
            lengths.append(sequence_of_snapshots.shape[0])
num_val = np.sum(lengths)
true_labels = []

# Compute Log Likelihoods for each
log_likelihoods = np.zeros((num_val, num_codes, num_time_steps))
for true_ATA6code in codes:
    # time steps 
    for true_k in val_map_revived[ATA6code].keys():
        # for sequence of snapshots
        for j,sample in enumerate(val_map_revived[true_ATA6code][true_k]):
             
            # get true labels
            true_labels = np.append(true_labels,[true_ATA6code,true_k])
            for i, ATA6code in enumerate(codes):
                for k in val_map_revived[ATA6code].keys():
                    x = sample.as_matrix()[:,2:] 
                    print(i,j,k)
                    log_likelihoods[j,i,k] = models[sub2ind([num_codes, num_time_steps],i,k)].score(x)
                    
#true_labels.reshape((num_val,2))

In [ ]:
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')


X1 = [[0.0,23.0,101.5, 0], [0.0,22.0,101.5, 0], [0.0,23,101.7, 0]]
X2 = [[0.0,24.68,102.0, 0], [0.0,24.5,103.5, 0.2], [0.0,25.5,103.2, 0.5], [0.0,26.5,103.2, 1]]
X3 = [[0.0,24.68,102.0, 0], [0.0,22.0,101.5, 0], [0.0,23,101.7, 0], [0.0,26.5,103.2, 1]]
X = np.concatenate([X1, X2, X3])
lengths = [len(X1), len(X2), len(X3)]

model = hmm.GaussianHMM(n_components=2, n_iter=1000).fit(X,lengths)
log_likelihood1 = model.score(X1)
log_likelihood2 = model.score(X2)
log_likelihood3 = model.score(X3)

log_likelihoods = [log_likelihood1, log_likelihood2, log_likelihood3]
probabilities = softmax(np.array(log_likelihoods).reshape((1,3)))
print("log likelihoods: ", log_likelihoods)
print("softmax probabilities: ", probabilities)